<a href="https://colab.research.google.com/github/ucalyptus/BS-Nets-Implementation-Pytorch/blob/master/HSI_SE_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install spectral
!pip install keras==2.2.4

     |████████████████████████████████| 153kB 2.8MB/s 
  Created wheel for spectral: filename=spectral-0.20-cp36-none-any.whl size=183918 sha256=1f82b4248a58d9fdf276ddd32041adda342035dc87d086f2c81bb146d04d9804
  Stored in directory: /root/.cache/pip/wheels/ad/cf/f3/3cab28f6aed46f15c8db09c6ad678483610426261025e61ff8
Successfully built spectral
     |████████████████████████████████| 317kB 2.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
import sys
import os
sys.path.insert(0, os.getcwd()+'/drive/My Drive/HSI/')

In [0]:
import tensorflow as tf
import scipy

import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, BatchNormalization, Activation 
from keras.layers import Add, Permute, Multiply, Dropout, Input, Concatenate, Reshape
from keras.layers import GlobalMaxPooling3D, GlobalAveragePooling3D, MaxPooling2D 
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import backend as K

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import spectral

%matplotlib inline

Using TensorFlow backend.


# Data Loading

In [0]:
## GLOBAL VARIABLES
dataset = 'IP'
test_ratio = 0.85
windowSize = 15

In [0]:
def loadData(name):
    data_path = os.path.join(os.getcwd()+'/drive/My Drive/HSI/','data')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
    
    return data, labels

In [0]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [0]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [0]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [0]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [0]:
X, y = loadData(dataset)

X.shape, y.shape

((145, 145, 200), (145, 145))

In [0]:
L = 30 if dataset == 'IP' else 15
X,pca = applyPCA(X,numComponents=L)

X.shape

(145, 145, 30)

In [0]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

((10249, 15, 15, 30), (10249,))

In [0]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

In [0]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, L, 1)
ytrain = np_utils.to_categorical(ytrain)

In [0]:
Xtest = Xtest.reshape(-1, windowSize, windowSize, L, 1)
ytest = np_utils.to_categorical(ytest)

In [0]:
Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((1537, 15, 15, 30, 1), (8712, 15, 15, 30, 1), (1537, 16), (8712, 16))

# CBOF


In [0]:
from keras.engine.topology import Layer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances
from keras.initializers import Constant

In [0]:
bof = 32

In [0]:
class BoF_Pooling(Layer):
    def __init__(self, n_codewords, spatial_level=0, **kwargs):
        self.N_k = n_codewords
        self.spatial_level = spatial_level
        self.V, self.sigmas = None, None

        super(BoF_Pooling, self).__init__(**kwargs)

    def build(self, input_shape):
        self.V = self.add_weight(name='codebook', shape=(1, 1, input_shape[3], self.N_k), initializer='uniform',
                                 trainable=True)
        self.sigmas = self.add_weight(name='sigmas', shape=(1, 1, 1, self.N_k), initializer=Constant(0.1),
                                      trainable=True)
        super(BoF_Pooling, self).build(input_shape)

    def call(self, x):
        # Calculate the pairwise distances between the codewords and the feature vectors
        x_square = K.sum(x ** 2, axis=3, keepdims=True)
        y_square = K.sum(self.V ** 2, axis=2, keepdims=True)
        dists = x_square + y_square - 2 * K.conv2d(x, self.V, strides=(1, 1), padding='valid')
        dists = K.maximum(dists, 0)

        # Quantize the feature vectors
        quantized_features = K.softmax(- dists / (self.sigmas ** 2))

        # Compile the histogram
        if self.spatial_level == 0:
            histogram = K.mean(quantized_features, [1, 2])
        elif self.spatial_level == 1:
            shape = K.shape(quantized_features)
            mid_1 = K.cast(shape[1] / 2, 'int32')
            mid_2 = K.cast(shape[2] / 2, 'int32')
            histogram1 = K.mean(quantized_features[:, :mid_1, :mid_2, :], [1, 2])
            histogram2 = K.mean(quantized_features[:, mid_1:, :mid_2, :], [1, 2])
            histogram3 = K.mean(quantized_features[:, :mid_1, mid_2:, :], [1, 2])
            histogram4 = K.mean(quantized_features[:, mid_1:, mid_2:, :], [1, 2])
            histogram = K.stack([histogram1, histogram2, histogram3, histogram4], 1)
            histogram = K.reshape(histogram, (-1, 4 * self.N_k))
        else:
            # No other spatial level is currently supported (it is trivial to extend the code)
            assert False

        # Simple trick to avoid rescaling issues
        return histogram * self.N_k

    def compute_output_shape(self, input_shape):
        if self.spatial_level == 0:
            return (input_shape[0], self.N_k)
        elif self.spatial_level == 1:
            return (input_shape[0], 4 * self.N_k)

# Model Imports

In [0]:
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    merge,
    Dense,
    Flatten,
    Dropout
)
from keras.layers.convolutional import (
    Convolution3D,
    MaxPooling3D,
    AveragePooling3D,
    Conv3D
)
#import cbof
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling3D
from keras.layers import GlobalMaxPooling2D
from keras.layers import GlobalMaxPooling3D

from keras.layers import Conv2D
from keras.layers import add

from keras.layers import maximum
from keras.layers import multiply
from keras.regularizers import l2
from keras.utils import conv_utils
from keras.utils.data_utils import get_file
from keras.engine.topology import get_source_inputs
from keras_applications.imagenet_utils import _obtain_input_shape
from keras_applications.resnet50 import preprocess_input
from keras_applications.imagenet_utils import decode_predictions
from keras.regularizers import l2
from keras import regularizers
from keras import backend as K

# SE-Resnet8 subfunctions

In [0]:
def _handle_dim_ordering():
    global CONV_DIM1
    global CONV_DIM2
    global CONV_DIM3
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        CONV_DIM1 = 1
        CONV_DIM2 = 2
        CONV_DIM3 = 3
        CHANNEL_AXIS = 4
    else:
        CHANNEL_AXIS = 1
        CONV_DIM1 = 2
        CONV_DIM2 = 3
        CONV_DIM3 = 4

def _bn_relu(input):
	norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
	return Activation("relu")(norm)

def _conv_bn_relu(**params):
    nb_filter = params["nb_filter"]
    kernel_dim1 = params["kernel_dim1"]
    kernel_dim2 = params["kernel_dim2"]
    kernel_dim3 = params["kernel_dim3"]
    subsample = params.setdefault("subsample", (1, 1, 1))
    init = params.setdefault("init", "he_normal")
    border_mode = params.setdefault("border_mode", "same")
    W_regularizer = params.setdefault("W_regularizer", regularizers.l2(1.e-4))
    def f(input):
        conv = Conv3D(kernel_initializer=init,strides=subsample,kernel_regularizer= W_regularizer, filters=nb_filter, kernel_size=(kernel_dim1,kernel_dim2,kernel_dim3))(input)
        return _bn_relu(conv)

    return f

def _bn_relu_conv(**params):
    nb_filter = params["nb_filter"]
    kernel_dim1 = params["kernel_dim1"]
    kernel_dim2 = params["kernel_dim2"]
    kernel_dim3 = params["kernel_dim3"]
    subsample = params.setdefault("subsample", (1,1,1))
    init = params.setdefault("init", "he_normal")
    border_mode = params.setdefault("border_mode", "same")
    W_regularizer = params.setdefault("W_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv3D(kernel_initializer=init, strides=subsample, kernel_regularizer=W_regularizer,
                          filters=nb_filter, kernel_size=(kernel_dim1, kernel_dim2, kernel_dim3), padding=border_mode)(activation)

    return f

def _shortcut(input, residual):
    stride_dim1 = (input._keras_shape[CONV_DIM1]+1) // residual._keras_shape[CONV_DIM1]
    stride_dim2 = (input._keras_shape[CONV_DIM2]+1) // residual._keras_shape[CONV_DIM2]
    stride_dim3 = (input._keras_shape[CONV_DIM3]+1) // residual._keras_shape[CONV_DIM3]
    equal_channels = residual._keras_shape[CHANNEL_AXIS] == input._keras_shape[CHANNEL_AXIS]

    shortcut = input
    print("input shape:", input._keras_shape)

    shortcut = Conv3D(kernel_initializer="he_normal", strides=(stride_dim1, stride_dim2, stride_dim3), kernel_regularizer=regularizers.l2(0.0001),filters=residual._keras_shape[CHANNEL_AXIS], kernel_size=(1, 1, 1), padding='valid')(input)
    #shortcut = channel_spatial_squeeze_excite(shortcut)
    shortcut = squeeze_excite_gap_block(shortcut)
    
    return add([shortcut, residual])
	
def _shortcut_spc(input, residual):
    stride_dim1 = (input._keras_shape[CONV_DIM1]+1) // residual._keras_shape[CONV_DIM1]
    stride_dim2 = (input._keras_shape[CONV_DIM2]+1) // residual._keras_shape[CONV_DIM2]
    stride_dim3 = (input._keras_shape[CONV_DIM3]+1) // residual._keras_shape[CONV_DIM3]
    equal_channels = residual._keras_shape[CHANNEL_AXIS] == input._keras_shape[CHANNEL_AXIS]

    shortcut = input
    print("input shape:", input._keras_shape)
    #shortcut = channel_spatial_squeeze_excite(residual)
    shortcut = squeeze_excite_gap_block(shortcut)
    return add([shortcut, residual])
	
	
def basic_block(nb_filter, init_subsample=(1, 1, 1), is_first_block_of_first_layer=False):
    def f(input):

        if is_first_block_of_first_layer:
            conv1 = Conv3D(kernel_initializer="he_normal", strides=init_subsample, kernel_regularizer=regularizers.l2(0.0001),
                          filters=nb_filter, kernel_size=(3, 3, 1), padding='same')(input)
        else:
            conv1 = _bn_relu_conv(nb_filter=nb_filter, kernel_dim1=3, kernel_dim2=3, kernel_dim3=1, subsample=init_subsample)(input)

        residual = _bn_relu_conv(nb_filter=nb_filter, kernel_dim1=3, kernel_dim2=3, kernel_dim3=1)(conv1)
        return _shortcut(input, residual)

    return f

def basic_block_spc(nb_filter, init_subsample=(1, 1, 1), is_first_block_of_first_layer=False):
    def f(input):

        if is_first_block_of_first_layer:
            conv1 = Conv3D(kernel_initializer="he_normal", strides=init_subsample, kernel_regularizer=regularizers.l2(0.0001),
                          filters=nb_filter, kernel_size=(3, 3, 1), padding='same')(input)
        else:
            conv1 = _bn_relu_conv(nb_filter=nb_filter, kernel_dim1=3, kernel_dim2=3, kernel_dim3=1, subsample=init_subsample)(input)

        residual = _bn_relu_conv(nb_filter=nb_filter, kernel_dim1=3, kernel_dim2=3, kernel_dim3=1)(conv1)
        return _shortcut_spc(input, residual)

    return f




# Squeeze Excitation Combined

In [0]:
def squeeze_excite_gap_block(input, ratio=16):
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling3D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)


    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x
def squeeze_excite_gmp_block(input, ratio=16):
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalMaxPooling3D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x


def channel_spatial_squeeze_excite(input, ratio=16):
    ''' Create a spatial squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
    Returns: a keras tensor
    References
    -   [Squeeze and Excitation Networks](https://arxiv.org/abs/1709.01507)
    -   [Concurrent Spatial and Channel Squeeze & Excitation in Fully Convolutional Networks](https://arxiv.org/abs/1803.02579)
    '''

    cse = squeeze_excite_gap_block(input, ratio)
    sse = squeeze_excite_gmp_block(input, ratio)
    
    x = maximum([cse, sse])   #Lots of funsion can be done line average, max etc.
    return x

# SE-Resnet8

In [0]:
def SE_ResNet8(input_shape,codebooks, num_outputs):
	_handle_dim_ordering()
	if len(input_shape) != 4:
		raise Exception("Input shape should be a tuple (nb_channels, kernel_dim1, kernel_dim2, kernel_dim3)")
	
	if K.common.image_dim_ordering() == 'tf':
		input_shape = (input_shape[1], input_shape[2],input_shape[3], input_shape[0])
	input = Input(shape = input_shape)#;print(input.shape)
	
	conv1 = _conv_bn_relu(nb_filter=32, kernel_dim1=1, kernel_dim2=1, kernel_dim3=7, subsample=(1, 1, 2))(input)#;print(conv1.shape)
	
	conv2 = basic_block_spc(32, is_first_block_of_first_layer = True)(conv1)#;print(conv2.shape)
	bn3 = _bn_relu(conv2)#;print(bn3.shape)
	#bn3 = _bn_relu(bn3_1)
	bn4 = _conv_bn_relu(nb_filter=128, kernel_dim1=1, kernel_dim2=1, kernel_dim3=bn3._keras_shape[CONV_DIM3], subsample=(1, 1, 2) , border_mode='valid')(bn3)
	resh = Reshape((bn4._keras_shape[CONV_DIM1],bn4._keras_shape[CONV_DIM2],bn4._keras_shape[CHANNEL_AXIS],1))(bn4)
	conv4 = _conv_bn_relu(nb_filter=32, kernel_dim1=3, kernel_dim2=3, kernel_dim3=128,
                              subsample=(1, 1, 1))(resh)
	conv5 = basic_block(32, is_first_block_of_first_layer = True)(conv4)#;print(conv5.shape)
	bn5 = _bn_relu(conv5)
	bn6 = _bn_relu(bn5)

	bn6 = Reshape((bn6.shape[1],bn6.shape[2],bn6.shape[4]))(bn6)
	#print(bn6.shape)
	#pool2 = AveragePooling3D(pool_size=(bn6._keras_shape[CONV_DIM1],
  #                                    bn6._keras_shape[CONV_DIM2],
  #                                    bn6._keras_shape[CONV_DIM3],),strides=(1, 1, 1))(bn6)
	#flatten1 = Flatten()(pool2)
	#bn6 = K.squeeze(bn6,axis = 3)
	flatten1 = BoF_Pooling(codebooks, spatial_level=0)(bn6)
	drop1 = Dropout(rate=0.5)(flatten1)
	dense = Dense(units=num_outputs, activation="softmax", kernel_initializer="he_normal")(drop1);print(dense.dtype,input.dtype)

	model = Model( inputs = input , outputs = dense)
	#model.summary()
	return model
model = SE_ResNet8((1,15,15,30),64,16)

input shape: (None, 15, 15, 12, 32)
input shape: (None, 13, 13, 1, 32)
<dtype: 'float32'> <dtype: 'float32'>


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 15, 15, 30, 1 0                                            
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 15, 15, 12, 3 256         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 15, 15, 12, 3 128         conv3d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 15, 15, 12, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
global_ave

In [0]:
def initialize_bof_layers(model, data, n_samples=100, n_feature_samples=5000, batch_size=32, k_means_max_iters=300,
                          k_means_n_init=4):
    #print(data.shape)
    for i in range(len(model.layers)):
        if isinstance(model.layers[i], BoF_Pooling):
            print("Found BoF layer (layer %d), initializing..." % i)
            cur_layer = model.layers[i]

            # Compile a function for getting the feature vectors
            get_features = K.function([model.input] + [K.learning_phase()], [model.layers[i].input])
            features = []
            for j in range(int(n_samples / batch_size)):
                cur_feats = get_features([data[j * batch_size:(j + 1) * batch_size], 0])[0]
                #print(cur_feats.shape)
                features.append(cur_feats.reshape((-1, cur_feats.shape[3])))
            features = np.concatenate(features)
            np.random.shuffle(features)
            features = features[:n_feature_samples]

            # Cluster the features
            kmeans = KMeans(n_clusters=cur_layer.N_k, n_init=k_means_n_init, max_iter=k_means_max_iters)
            kmeans.fit(features)
            V = kmeans.cluster_centers_.T
            V = V.reshape((1, 1, V.shape[0], V.shape[1]))

            # Set the value for the codebook
            K.set_value(cur_layer.V, np.float32(V))

            # Get the mean distance for initializing the sigmas
            mean_dist = np.mean(pairwise_distances(features[:100]))

            # Set the value for sigmas
            sigmas = np.ones((1, 1, 1, cur_layer.N_k)) * (mean_dist ** 2)
            K.set_value(cur_layer.sigmas, np.float32(sigmas))

# Training

In [0]:
# compiling the model
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
initialize_bof_layers(model, Xtrain,n_samples=Xtrain.shape[0])

Found BoF layer (layer 39), initializing...


In [0]:
# checkpoint
filepath = "best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]

In [0]:
history = model.fit(x=Xtrain, y=ytrain, batch_size= 32, epochs=200 ,callbacks=callbacks_list)

Epoch 1/200
1537/1537 [==============================] - 4s 3ms/step - loss: 3.6697 - acc: 0.0891

Epoch 00001: acc improved from -inf to 0.08913, saving model to best-model.hdf5


TypeError: ignored

# Save Model

In [0]:
model.save_weights('my_model.h5')

# Validation

In [0]:
# load best weights
#model.load_weights("best-model.hdf5")
#model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
Xtest.shape

In [0]:
Y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

In [0]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [0]:
Y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

In [0]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [0]:
def reports (X_test,y_test,name):
    #start = time.time()
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    #end = time.time()
    #print(end - start)
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']
    
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100

In [0]:
classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(Xtest,ytest,dataset)
classification = str(classification)
confusion = str(confusion)
file_name = "classification_report_"+"dataset="+dataset+"_window_size="+str(windowSize)+"_test_ratio="+str(test_ratio)+".txt"

with open(file_name, 'w') as x_file:
    x_file.write('{} Test loss (%)'.format(Test_loss))
    x_file.write('\n')
    x_file.write('{} Test accuracy (%)'.format(Test_accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))
    x_file.write('\n')
    x_file.write('{}Class-wise accuracy (%)'.format(each_acc))

In [0]:
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    
    return patch

In [0]:
# load the original image
X, y = loadData(dataset)

In [0]:
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = windowSize
numComponents = 30

In [0]:
X,pca = applyPCA(X, numComponents=numComponents)

In [0]:
X = padWithZeros(X, PATCH_SIZE//2)

In [0]:
im1 = np.zeros((1,25,25,30,1))
# calculate the predicted image
outputs = np.zeros((height,width))
for i in range(height):
    for j in range(width):
        target = int(y[i,j])
        if target == 0 :
            continue
        else :
            image_patch=Patch(X,i,j)
            #print(image_patch.shape)
            X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1], image_patch.shape[2],1).astype('float32')                                   
            
            if i==20 and j==20:
              im1 = X_test_image
              
            prediction = (model.predict(X_test_image))
            prediction = np.argmax(prediction, axis=1)
            outputs[i][j] = prediction+1

In [0]:
ground_truth = spectral.imshow(classes = y,figsize =(7,7))

In [0]:
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))

In [0]:
spectral.save_rgb("predictions_IP_25.jpg", outputs.astype(int), colors=spectral.spy_colors)